In [ ]:
import streamlit as st
import sqlite3
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load model once
@st.cache_resource
def load_model():
    return SentenceTransformer("all-MiniLM-L6-v2")

model = load_model()

# Connect to SQLite
conn = sqlite3.connect("your_data.db")  # 🔁 Update path if needed
df = pd.read_sql_query("SELECT id, title, abstract FROM patents", conn)
df["text"] = df["title"].fillna("") + ". " + df["abstract"].fillna("")
texts = df["text"].tolist()

# Create embeddings
@st.cache_resource
def create_index(texts):
    embeddings = model.encode(texts, show_progress_bar=True)
    dim = embeddings[0].shape[0]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, embeddings

index, embeddings = create_index(texts)

# Streamlit UI
st.title("🔎 Patent Semantic Search Engine")
st.write("Search patents by meaning using AI-powered semantic search.")

query = st.text_input("Enter your search query:")
top_k = st.slider("Number of results", 1, 10, 3)

if st.button("Search") and query:
    query_vector = model.encode([query])
    distances, indices = index.search(np.array(query_vector), top_k)

    st.markdown(f"### 🔍 Top {top_k} Results for: `{query}`")
    for i in indices[0]:
        st.markdown("----")
        st.subheader(df.iloc[i]["title"])
        st.write(df.iloc[i]["abstract"])
